In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

sb.set()

# Data Collection
We'll be collecting the following data from the following sites, between the periods of Apr 2013 and Mar 2023.
- Rainfall
- Temperature

(http://www.weather.gov.sg)

- Humidity

(https://www.wunderground.com)

Some limitations/constraints when collecting our data:

CSV files from weather.gov.sg are given in monthly format. However, there will be too much csv files to deal with if we concatenate them one by one in the notebook. Therefore we manually pasted monthly values into one csv file for each year

The humidity data from wunderground.com was presented in HTML tables. However, due to some unknown formatting issues on that website, we could not extract the HTML table using pandas's read_html() function. Therefore we manually extracted the data from the tables and pasted them in the yearly csv files.

In [13]:
rain13 = pd.read_csv('data/2013.csv', encoding = "ISO-8859-1") #Parameter required due to unknown format error in csv file
rain14 = pd.read_csv('data/2014.csv', encoding = "ISO-8859-1") #Parameter required due to unknown format error in csv file
rain15 = pd.read_csv('data/2015.csv', encoding = "ISO-8859-1") #Parameter required due to unknown format error in csv file
rain16 = pd.read_csv('data/2016.csv', encoding = "ISO-8859-1") #Parameter required due to unknown format error in csv file
rain17 = pd.read_csv('data/2017.csv', encoding = "ISO-8859-1") #Parameter required due to unknown format error in csv file
rain18 = pd.read_csv('data/2018.csv', encoding = "ISO-8859-1") #Parameter required due to unknown format error in csv file
rain19 = pd.read_csv('data/2019.csv', encoding = "ISO-8859-1") #Parameter required due to unknown format error in csv file
rain20 = pd.read_csv('data/2020.csv')
rain21 = pd.read_csv('data/2021.csv')
rain22 = pd.read_csv('data/2022.csv')
rain23 = pd.read_csv('data/2023.csv')
rainData = pd.concat([rain13, rain14, rain15, rain16, rain17, rain18, rain19, rain20, rain21, rain22, rain23])
rainData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3652 entries, 0 to 89
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Station                        3652 non-null   object 
 1   Year                           3652 non-null   int64  
 2   Month                          3652 non-null   int64  
 3   Day                            3652 non-null   int64  
 4   Daily Rainfall Total (mm)      3652 non-null   float64
 5   Highest 30 min Rainfall (mm)   3652 non-null   object 
 6   Highest 60 min Rainfall (mm)   3652 non-null   object 
 7   Highest 120 min Rainfall (mm)  3652 non-null   object 
 8   Mean Temperature (°C)          3652 non-null   float64
 9   Maximum Temperature (°C)       3652 non-null   float64
 10  Minimum Temperature (°C)       3652 non-null   float64
 11  Mean Wind Speed (km/h)         3652 non-null   object 
 12  Max Wind Speed (km/h)          3652 non-null   obj

# Data Cleaning
There are several columns that we'll not use for our prediction. For e.g,

- Station

All records were retrieved from Changi Weather Station, hence we do not need this column.

- Highest 30 min Rainfall (mm)
- Highest 60 min Rainfall (mm)
- Highest 120 min Rainfall (mm)

It does not make sense to use these rainfall values as our predictors, as we want to predict our total rainfall using datasets independent from rainfall.

- Maximum Temperature (°C)
- Minimum Temperature (°C)

We'll be using Mean Temperature as one of our predictors, making these unncessary.

- Mean Wind Speed (km/h)
- Max Wind Speed (km/h)

Wind Speed theoretically has no relationship with total rainfall amount, making these columns irrelevant

Let's remove these columns to make our EDA and Machine Learning later easier.

In [15]:
rainData = rainData[['Year','Month','Day','Daily Rainfall Total (mm)','Mean Temperature (°C)','Mean Humidity (%)']]

rainData['Date'] = rainData['Year'].astype(str) + '-' + rainData['Month'].astype(str) + '-' + rainData['Day'].astype(str)
rainData['Date'] = pd.to_datetime(rainData['Date'], infer_datetime_format=True)
rainData = rainData.set_index('Date')
rainData = rainData.sort_index()

rainData['Month'] = rainData.index.month
rainData['Year'] = rainData.index.year
rainData.tail().style.hide(axis='index')

Year,Month,Day,Daily Rainfall Total (mm),Mean Temperature (°C),Mean Humidity (%)
2023,3,27,19.400000,26.000000,90.700000
2023,3,28,0.000000,28.800000,80.600000
2023,3,29,0.000000,28.800000,79.700000
2023,3,30,9.200000,28.400000,80.300000
2023,3,31,0.000000,28.700000,80.100000


In [16]:
rainData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3652 entries, 2013-04-01 to 2023-03-31
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       3652 non-null   int64  
 1   Month                      3652 non-null   int64  
 2   Day                        3652 non-null   int64  
 3   Daily Rainfall Total (mm)  3652 non-null   float64
 4   Mean Temperature (°C)      3652 non-null   float64
 5   Mean Humidity (%)          3652 non-null   float64
dtypes: float64(3), int64(3)
memory usage: 199.7 KB


Thankfully we do not have any null values in our records after removing unnecessary columns. 
To be safe, let's check if there are any cells with invalid (i.e. negative) values that would affect our EDA and Machine Learning later on.

In [17]:
print("Rainfall with Negative Value: ", len(rainData[rainData['Daily Rainfall Total (mm)'] < 0]))
print("Temperature with Negative Value: ", len(rainData[rainData['Mean Temperature (°C)'] < 0]))
print("Humidity with Negative Value: ", len(rainData[rainData['Mean Humidity (%)'] < 0]))

Rainfall with Negative Value:  0
Temperature with Negative Value:  0
Humidity with Negative Value:  0


Finally, Let's export our cleaned data to be used by our EDA and Machine Learning Sections later.

In [18]:
rainData.to_csv('data/rainData.csv') 